## Лабораторная работа 3
ИУ5-21М Стельмах Яна
## Цель работы 
Изучение продвинутых способов предварительной обработки данных для дальнейшего формирования моделей.

## Задание
Выбрать один или несколько наборов данных (датасетов) для решения следующих задач. Каждая задача может быть решена на отдельном датасете, или несколько задач могут быть решены на одном датасете. Просьба не использовать датасет, на котором данная задача решалась в лекции.

Для выбранного датасета (датасетов) на основе материалов лекций решить следующие задачи:

масштабирование признаков (не менее чем тремя способами);

обработку выбросов для числовых признаков (по одному способу для удаления выбросов и для замены выбросов);

обработку по крайней мере одного нестандартного признака (который не является числовым или категориальным);

отбор признаков:

один метод из группы методов фильтрации (filter methods);

один метод из группы методов обертывания (wrapper methods);

один метод из группы методов вложений (embedded methods).

## Ход работы

In [1]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
import pandas as pd


file_path = '/Users/Iana/Desktop/2М/MMO/data/wine+quality/winequality-red.csv'

# Загрузка данных
wine = pd.read_csv(file_path, sep=';')  # Указываем разделитель ';'

Масштабирование признаков (3 способа)

In [ ]:
# 1. Стандартизация (Z-score)
scaler = StandardScaler()
data_scaled1 = wine.copy()  # Создаем копию DataFrame
data_scaled1[['alcohol', 'residual sugar']] = scaler.fit_transform(wine[['alcohol', 'residual sugar']])

# 2. Min-Max Scaling
scaler = MinMaxScaler()
data_scaled2 = wine.copy()
data_scaled2[['alcohol', 'residual sugar']] = scaler.fit_transform(wine[['alcohol', 'residual sugar']])

# 3. Robust Scaling (устойчивый к выбросам)
scaler = RobustScaler()
data_scaled3 = wine.copy()
data_scaled3[['alcohol', 'residual sugar']] = scaler.fit_transform(wine[['alcohol', 'residual sugar']])

# Проверка результатов
print("Стандартизация:")
print(data_scaled1[['alcohol', 'residual sugar']].head())
print("\nMin-Max Scaling:")
print(data_scaled2[['alcohol', 'residual sugar']].head())
print("\nRobust Scaling:")
print(data_scaled3[['alcohol', 'residual sugar']].head())

Стандартизация:
    alcohol  residual sugar
0 -0.960246       -0.453218
1 -0.584777        0.043416
2 -0.584777       -0.169427
3 -0.584777       -0.453218
4 -0.960246       -0.453218

Min-Max Scaling:
    alcohol  residual sugar
0  0.153846        0.068493
1  0.215385        0.116438
2  0.215385        0.095890
3  0.215385        0.068493
4  0.153846        0.068493

Robust Scaling:
   alcohol  residual sugar
0    -0.50       -0.428571
1    -0.25        0.571429
2    -0.25        0.142857
3    -0.25       -0.428571
4    -0.50       -0.428571


Обработка выбросов методом Тьюки

In [ ]:
import chardet

file_path = '/Users/Iana/Desktop/2М/MMO/data/Superstore.csv'

# Загружаем с правильной кодировкой
store = pd.read_csv(file_path, encoding='ISO-8859-1', delimiter=',')

In [ ]:
# Удаление выбросов (метод Тьюки)
def remove_outliers(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    return df[(df[col] >= Q1 - 1.5*IQR) & (df[col] <= Q3 + 1.5*IQR)]

store_clean = remove_outliers(store, 'Sales')

# Замена выбросов (Winsorization)
from scipy.stats.mstats import winsorize
store['Sales_winsorized'] = winsorize(store['Sales'], limits=[0.05, 0.05])

Обработка нестандартного признака

In [ ]:
file_path = '/Users/Iana/Desktop/2М/MMO/data/movies.csv'

# Загрузка данных
movies = pd.read_csv(file_path) 

In [15]:
# Обработка JSON-поля 'genres'
movies['genres'] = movies['genres'].apply(lambda x: [i['name'] for i in eval(x)])
movies['main_genre'] = movies['genres'].apply(lambda x: x[0] if x else None)

Отбор признаков

In [19]:
import pandas as pd
from sklearn.feature_selection import (SelectKBest, f_classif, 
                                     RFE, SelectFromModel)
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

file_path = '/Users/Iana/Desktop/2М/MMO/data/cancer.csv'

# Загрузка данных
cancer = pd.read_csv(file_path) 

In [20]:
# Подготовка данных
# Предполагаем, что целевая переменная называется 'target' или 'diagnosis'
# и остальные колонки - это признаки
X = cancer.drop(['id', 'diagnosis', 'Unnamed: 32'], axis=1, errors='ignore')  # Удаляем ненужные колонки
y = cancer['diagnosis']  # Или cancer['target'], в зависимости от датасета

# Кодируем целевую переменную, если она строковая
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

# 1. Filter Method: ANOVA F-value
selector = SelectKBest(f_classif, k=5)
X_new = selector.fit_transform(X, y)
print("Выбранные признаки (ANOVA):", X.columns[selector.get_support()])

# 2. Wrapper Method: Recursive Feature Elimination
estimator = SVC(kernel="linear")
selector_rfe = RFE(estimator, n_features_to_select=3, step=1)
X_rfe = selector_rfe.fit_transform(X, y)
print("Выбранные признаки (RFE):", X.columns[selector_rfe.get_support()])

# 3. Embedded Method: Feature Importance from Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X, y)
selector_emb = SelectFromModel(model, threshold='median')
X_emb = selector_emb.fit_transform(X, y)
print("Выбранные признаки (Random Forest):", X.columns[selector_emb.get_support()])

Выбранные признаки (ANOVA): Index(['perimeter_mean', 'concave points_mean', 'radius_worst',
       'perimeter_worst', 'concave points_worst'],
      dtype='object')
Выбранные признаки (RFE): Index(['concavity_worst', 'concave points_worst', 'symmetry_worst'], dtype='object')
Выбранные признаки (Random Forest): Index(['radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean',
       'concavity_mean', 'concave points_mean', 'radius_se', 'area_se',
       'radius_worst', 'texture_worst', 'perimeter_worst', 'area_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst'],
      dtype='object')
